<a href="https://colab.research.google.com/github/mahnoorcheema/Dog-Breed-Identification/blob/master/CSI4106_Dog_Breed_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Dog Breed Identification Challenge

## Install Kaggle 

In [80]:
!pip install kaggle

##Import Libraries

In [0]:
import os
import time
from datetime import timedelta
import math
import tensorflow as tf
import scipy.misc
from random import sample
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from IPython.display import display
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

##Kaggle Pre-Processing
Link to account and download files (training set, test set, labels)

**Delete for Submission**

In [82]:
#Links to account
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"mahnoorcheema","key":"51dd5a6107bb02e5d0a2ec3e15393a68"}'}

In [83]:
#Checks to see if the file is in the directory
ls -lha kaggle.json

-rw-r--r-- 1 root root 69 Nov 21 20:36 kaggle.json


In [84]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#Downloads data from Kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dog-breed-identification

labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)


##Normalize Data

In [0]:
#Uncompress Train and Test files
archive_train = ZipFile("train.zip", 'r')
archive_test = ZipFile("test.zip", 'r')

In [0]:
# Creates pickle file of images 
###############
def normalize_data(archivezip, width, height, save_name):
     #We choose the archive (zip file) + the new width and height for all the image which will be reshaped
    s = (len(archivezip.namelist()[:])-1, width, height,3) #width x height = number of features because images are width x height pixels
    allImage = np.zeros(s)

    for i in range(1,len(archivezip.namelist()[:])):
        filename = BytesIO(archivezip.read(archivezip.namelist()[i]))
        image = PIL.Image.open(filename) # open colour image
        image = image.resize((width, height))
        image = np.array(image)
        image = np.clip(image/255.0, 0.0, 1.0) #255 = max of the value of a pixel

        allImage[i-1]=image
    
    #we save the newly created data base
    pickle.dump(allImage, open( save_name + '.p', "wb" ) )

In [110]:
image_resize = 60
#creates pickle file, this step can take several minutes
start = time.time()
normalize_data(archivezip = archive_train, 
               width = image_size, 
               height = image_size,
               save_name = "train")
normalize_data(archivezip = archive_test,
               width = image_size,
               height = image_size,
               save_name = "test")
end = time.time()
print("Time Taken:",end-start)

Time Taken: 113.03454422950745


In [88]:
#load training data and show it's shape
train = pickle.load( open( "train.p", "rb" ) )
print("The shape of the training data is: " , train.shape)

#load test data and show it's shape
test = pickle.load( open( "test.p", "rb" ) )
print("The shape of the test data is: " , test.shape)


The shape of the training data is:  (10222, 60, 60, 3)
The shape of the test data is:  (10357, 60, 60, 3)


In [0]:
#Retrieve labels from the CSV file provided
labels_raw = pd.read_csv("labels.csv.zip", 
                         compression='zip', 
                         header=0, sep=',', 
                         quotechar='"')

In [0]:
def itemfreq(matrix):
  """
  To accomodate for scipy.stats.itemfreq method being deprecated
  this method duplicates its functionality with non-deprecated function calls
  """
  return np.rot90(np.unique(matrix, return_counts=True))

In [98]:
number_of_breeds = 10
def reduce_dataset(labels_raw, number_of_breeds=10):
  """
  Get the 10 breeds that have the most occurences
  """
  #count occurence of each label
  frequent_labels = itemfreq(labels_raw["breed"])
  #sort in descending order
  frequent_labels = frequent_labels[frequent_labels[:, 1].argsort()[::-1]]
  #choose amount of breeds to be used for training
  main_labels = frequent_labels[:,0][0:number_of_breeds] 
  #transform labels to numpy matrix
  labels_matrix = labels_raw["breed"].as_matrix()
  labels_matrix = labels_matrix.reshape(labels_matrix.shape[0],1)

  labels_filtered = np.where(labels_matrix == main_labels)
  return labels_filtered

labels_reduced = reduce_dataset(labels_raw,number_of_breeds)
labels_filtered = labels_raw.iloc[labels_reduced[0],:]
train_filtered = train[labels_reduced[0],:,:,:]

print('- Number of images remaining after selecting the {0} main breeds : {1}'
      .format(number_of_breeds, labels_reduced[0].shape))
print('- The shape of train_filtered dataset is : {0}'
      .format(train_filtered.shape))


- Number of images remaining after selecting the 10 main breeds : (1141,)
- The shape of train_filtered dataset is : (1141, 60, 60, 3)


##Label encoder
Currently, our labels are text. We cannot train our model with data that contains text.

In [99]:
#We select the labels from the N main breeds
labels = labels_filtered["breed"].as_matrix()
labels = labels.reshape(labels.shape[0],1) #labels.shape[0] looks faster than using len(labels)
labels.shape

(1141, 1)

In [0]:
#Function to create one-hot labels
def matrix_Bin(labels):
    labels_bin=np.array([])

    labels_name, labels0 = np.unique(labels, return_inverse=True)
    labels0
    
    for _, i in enumerate(itemfreq(labels0)[:,0].astype(int)):
        labels_bin0 = np.where(labels0 == itemfreq(labels0)[:,0][i], 1., 0.)
        labels_bin0 = labels_bin0.reshape(1,labels_bin0.shape[0])

        if (labels_bin.shape[0] == 0):
            labels_bin = labels_bin0
        else:
            labels_bin = np.concatenate((labels_bin,labels_bin0 ),axis=0)

    print("Nber SubVariables {0}".format(itemfreq(labels0)[:,0].shape[0]))
    labels_bin = labels_bin.transpose()
    print("Shape : {0}".format(labels_bin.shape))
    
    return labels_name, labels_bin
#TODO: NEEDS TO RETURN IN DOUBLE LIST, NOT ARRAY
def get_identity_matrix(length):
  """Create One-Hot-Labels"""
  matrix = np.zeros((length, length))
  for index in range(length):
      matrix[index, index] = 1
  return matrix

In [105]:
#labels_name = np.unique(labels[:, 0])
#labels_bin = get_identity_matrix(len(labels_name))
labels_name,labels_bin = matrix_Bin(labels)
print(labels_name, labels_bin)
print(matrix_Bin(labels))

Nber SubVariables 10
Shape : (1141, 10)
['afghan_hound' 'basenji' 'bernese_mountain_dog' 'entlebucher'
 'great_pyrenees' 'maltese_dog' 'pomeranian' 'samoyed'
 'scottish_deerhound' 'shih-tzu'] [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Nber SubVariables 10
Shape : (1141, 10)
(array(['afghan_hound', 'basenji', 'bernese_mountain_dog', 'entlebucher',
       'great_pyrenees', 'maltese_dog', 'pomeranian', 'samoyed',
       'scottish_deerhound', 'shih-tzu'], dtype=object), array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]]))


##CNN
In this step, I create a convolutional neural network from scratch

In [106]:
num_validation = 0.30
X_train, X_validation, y_train, y_validation = train_test_split(train_filtered,
                                                                labels_bin,
                                                                test_size=num_validation,
                                                                random_state=6)
X_train.shape

(798, 60, 60, 3)

In [0]:
#from the one-hot database we can find the breed.
df_validation_toPred_cls = np.argmax(y_validation, axis=1)
# Our images are 60 pixels in each dimension.
image_size = image_resize
# RGB images, so 3 colour channels
num_channels = 3
# Images are stored in one-dimensional arrays of this length.
image_size_flat = image_size * image_size
# Image Shape
image_shape = (image_size, image_size, num_channels)
# Number of classes : 5 breeds
number_of_classes = number_of_breeds

*The following are methods to be used in the creation of the CNN.*

For the conv2d method,  the first and last stride must always be 1, because the first is for the image-number and the last is for the input-channel.
The padding is set to 'SAME' which means the input image is padded with zeroes so the size of the output is the same.
We are using a 2x2 max-pooling, which means that we consider 2x2 windows and select the largest value in each window. Then we move 2 pixels to the next window.
The Rectified Linear Unit (ReLU) layer calculates max(x, 0) for each input pixel x. This adds some non-linearity to the formula and allows us to learn more complicated functions.

In [0]:
def create_weights(shape):
  """
  returns random value from a truncated normal distribution
  """
  return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(length):
  """
  returns constant 0.05 as bias
  """
  return tf.Variable(tf.constant(0.05, shape=[length]))

def create_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=True,
                   use_dropout=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    weights = create_weights(shape=shape)
    biases = create_biases(length=num_filters)

    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)
    
    if use_dropout:
        layer = tf.nn.dropout(layer,keep_prob_conv)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights